In [ ]:
# Create a panel for generating code labs
from code_lab_generator import CodeLabGenerator
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

class LabPanel:
    def __init__(self):
        self.code_lab_generator = CodeLabGenerator()

        self._setup_ui()
        self._apply_styles()
        display(self.container)

    def _setup_ui(self):
        self.lang_dropdown = widgets.Dropdown(
            options=['Python', 'Java', 'JavaScript', 'TypeScript', 'C', 'C#', 'C++'],
            value='Python',
            description='Language:'
        )
        self.template_type = widgets.Dropdown(
            options=['function', 'class'],
            value='function',
            description='Type:'
        )
        self.name_text = widgets.Text(
            value='',
            placeholder='problem_name',
            description='Name:'
        )
        self.problem_text = widgets.Textarea(
            value='',
            description='Description:',
            layout={'width': '90%', 'height': '100px'}
        )
        self.params_text = widgets.Text(
            value='',
            placeholder='param1, param2',
            description='Parameters:'
        )
        self.generate_btn = widgets.Button(description='Generate')
        self.generate_btn.on_click(self._on_generate)
        self.status_display = widgets.HTML(value="")
        self.output = widgets.Output()
        self.container = widgets.VBox([
            self.lang_dropdown,
            self.template_type,
            self.name_text,
            self.problem_text,
            self.params_text,
            self.generate_btn,
            self.status_display,
            self.output
        ])

        self.lang_dropdown.observe(self._update_template_types, names='value')

    def _apply_styles(self):
        layout_style = widgets.Layout(
            width='auto',
            margin='10px', 
            border='1px solid #fff',
        )
        container_layout = widgets.Layout(
            width='100%',
            padding='10px',
        )
        style = """
            <style>
            .widget-label {
                color: #fff !important;
                background-color: #2D2D2D !important;
            }
            .jupyter-widgets {
                background-color: #2D2D2D !important;
                color: #fff !important;
            }
            .jp-OutputArea-output pre {
                color: #fff !important;
            }
            .error-message {
                color: #ff7675 !important;
                font-weight: bold;
                padding: 5px;
                margin-top: 10px;
            }
            .success-message {
                color: #00b894 !important;
                font-weight: bold;
                padding: 5px;
                margin-top: 10px;
            }
            </style>
        """

        for widget in [self.lang_dropdown, self.template_type, self.name_text, 
                    self.problem_text, self.params_text, self.generate_btn]:
            widget.layout = layout_style
        
        self.container = widgets.VBox([
            self.lang_dropdown,
            self.template_type,
            self.name_text,
            self.problem_text,
            self.params_text,
            self.generate_btn,
            self.status_display,
            self.output
        ], layout=container_layout)
        
        display(HTML(style))

    def _update_template_types(self, change):
            language = change['new']
            available_templates = self.code_lab_generator.templates.get(language, {})
            template_options = list(available_templates.keys())
            self.template_type.options = template_options
            
            if self.template_type.value not in template_options:
                self.template_type.value = template_options[0]

    def _validate_inputs(self):
        errors = []
        
        if not self.name_text.value.strip():
            errors.append("Name field cannot be empty")
        elif ' ' in self.name_text.value:
            errors.append("Name should not contain spaces")
        
        if not self.problem_text.value.strip():
            errors.append("Description field cannot be empty")
        
        return errors

    def _show_status(self, message, status_type="info"):
        if status_type == "error":
            self.status_display.value = f'<div class="error-message">{message}</div>'
        
        if status_type == "success":
            self.status_display.value = f'<div class="success-message">{message}</div>'

    def _on_generate(self, _):
        with self.output:
            clear_output()
        
        validation_errors = self._validate_inputs()

        if validation_errors:
            error_msg = "<br>".join([f"• {error}" for error in validation_errors])
            self._show_status(f"Please fix the following errors:<br>{error_msg}", "error")
            return
        
        with self.output:
            try:
                language = self.lang_dropdown.value
                name = self.name_text.value.strip()
                params = self.params_text.value.strip()
                desc = self.problem_text.value.strip()
                template_type = self.template_type.value 
                code = self.code_lab_generator.generate_code(
                    language,
                    name,
                    params,
                    desc,
                    template_type
                )
                
                self._show_status(f"Successfully generated {language} {template_type}", "success")
                print(f'Generated {language} {template_type}:')
                print(f'\n```{language.lower()}')
                print(code)
                print('```')
            except Exception as e:
                import traceback

                error_details = traceback.format_exc()

                self._show_status(f"Unexpected error: {str(e)}", "error")
                print(f"\nError: {str(e)}")
                print("\nDetailed error information:")
                print(error_details)

panel = LabPanel()

In [ ]:
# Test code lab generation
import io
import sys

test_inputs = [
    ('Python', 'add', 'a: int, b: int', 'Add two numbers', 'function'),
    ('Java', 'multiply', 'int x, int y', 'Multiply two numbers', 'class'),
    ('JavaScript', 'divide', 'num1, num2', 'Divide two numbers', 'function'),
    ('TypeScript', 'concat', 'str1: string, str2: string', 'Concatenate two strings', 'class'),
    ('C', 'subtract', 'int x, int y', 'Subtract two numbers', 'function'),
    ('C#', 'modulus', 'int a, int b', 'Modulus of two numbers', 'class'),
    ('C++', 'power', 'int base, int exp', 'Power function', 'function'),
]

for lang, name, params, desc, type in test_inputs:
    panel.lang_dropdown.value = lang
    panel.name_text.value = name
    panel.params_text.value = params
    panel.problem_text.value = desc
    panel.template_type.value = type
    buffer = io.StringIO()
    original_stdout = sys.stdout
    sys.stdout = buffer
    panel.generate_btn.click()
    output = buffer.getvalue()
    sys.stdout = original_stdout
    assert output, f"Output didn't change for {lang} {name}"